In [1]:
import tables
import pandas as pd
import io

In [2]:
%%bash
ls data

category.csv
question_category_train.csv
question_train.csv


## Clean category.csv

In [3]:
with open("data/category.csv", "r") as f:
    data = f.read()

In [4]:
cleaned = data.replace("\n\\\n", "~~~NEWLINE~~~")
cleaned = cleaned.replace('\\"', "~~~QUOTE~~~")

In [5]:
df = pd.read_csv(io.StringIO(cleaned))

In [6]:
df["description"] = df.description.fillna("").apply(lambda x: x.replace("~~~QUOTE~~~", '"').replace("~~~NEWLINE~~~", "\n"))

In [7]:
df.to_hdf("data/category.hdf5", "/df")

/Users/kayibal/virtualenvs/ml_project/lib/python3.5/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['category_name', 'headline', 'description', 'meta_description', 'meta_tag', 'url', 'created_at', 'updated_at']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [94]:
df2 = pd.read_csv("data/question_category_train.csv", parse_dates=["created_at"])

In [15]:
df.to_hdf("data/question_category_train.hdf5", "/df")

In [96]:
with open("data/question_train.csv", "r") as f:
    data = f.read()

In [97]:
def remove_quoted_newline(data):
    in_quotes = 0
    string = []
    for c in data:
        if c == '"' and in_quotes == 0:
            in_quotes +=1
            string.append(c)
        elif c == '"' and in_quotes == 1:
            in_quotes -= 1
            string.append(c)
        elif c == '\n' and in_quotes == 1:
            string.append("~~~NEWLINE~~~")
        else:
            string.append(c)
    if in_quotes > 0:
        print("unequal quote count")
    return "".join(string)

In [98]:
no_quotes = data.replace('\\"', "~~~QUOTE~~~")
cleaned = remove_quoted_newline(no_quotes)

In [99]:
for line in cleaned.splitlines():
    if '"9984"' in line:
        print(line)

"9984","0","0","70","","","Casting, Wettbewerb, Talent, Gesang","Prominente","stars-promis/prominente/","0","0","0","0","0","1","0","0","0000-00-00 00:00:00","8","2008-10-19 15:22:00","2010-04-12 15:21:43"


In [100]:
df = pd.read_csv(io.StringIO(cleaned), quotechar='"', sep=',')

In [101]:
df["question"] = df.question.fillna("").apply(lambda x: x.replace("~~~NEWLINE~~~", "\n").replace("~~~QUOTE~~~", '"'))

In [102]:
df.to_hdf("data/question_train.hdf5", "/df")

/Users/kayibal/virtualenvs/ml_project/lib/python3.5/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['category_main_id', 'question', 'description', 'tags', 'categories', 'url', 'editor_locked', 'created_at', 'updated_at']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
